<a href="https://colab.research.google.com/github/wina0901/mission/blob/main/%EB%AF%B8%EC%85%983_1%ED%8C%80_%EA%B9%80%EA%B8%B0%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. 미션 소개 및 주제

### 미션 목표

이번 미션의 최종 목표는 RMSLE (Root Mean Squared Logarithmic Error)를 최대한 낮추는 것입니다.

다양한 머신러닝 모델과 전략을 실험하여 가장 정확한 수요 예측 모델을 개발하고, 그 결과를 보고서에 담아 보세요!

-> 보고서를 따로 만들지 말고, 노트북을 잘 정리하기

### 미션 방향성


먼저 EDA와 데이터 전처리를 통하여, 데이터를 다듬고, 데이터를 분석한다.

그 다음, 배운 기본 지도 학습 알고리즘인 선형회귀, 다중회귀, 로지스틱회귀(제외), L1, L2정규화를 이용하여

RMSLE를 비교하며 적합한 모델을 직접 확인해보고,

하이퍼 파라미터를 통하여 직접 확인한 수치가 맞는지 비교를 한다.

최종적으로, 적합한 수요 예측 모델로 test 결과를 예측한다.

### 데이터 컬럼 설명

| 컬럼명          | 데이터 타입   | 설명                                                |
| ------------ | -------- | ------------------------------------------------- |
| `datetime`   | datetime | 자전거 대여 기록의 날짜 및 시간<br>예: `2011-01-01 00:00:00`    |
| `season`     | int      | 계절<br>(1: 봄, 2: 여름, 3: 가을, 4: 겨울)                 |
| `holiday`    | int      | 공휴일 여부<br>(0: 평일, 1: 공휴일)                         |
| `workingday` | int      | 근무일 여부<br>(0: 주말/공휴일, 1: 근무일)                     |
| `weather`    | int      | 날씨 상황<br>(1: 맑음, 2: 구름낌/안개, 3: 약간의 비/눈, 4: 폭우/폭설) |
| `temp`       | float    | 실측 온도 (섭씨)                                        |
| `atemp`      | float    | 체감 온도 (섭씨)                                        |
| `humidity`   | int      | 습도 (%)                                            |
| `windspeed`  | float    | 풍속 (m/s)                                          |
| `casual`     | int      | 등록되지 않은 사용자의 대여 수                                 |
| `registered` | int      | 등록된 사용자의 대여 수                                     |
| `count`      | int      | **총 대여 수 (종속 변수)**                                |


### 기본 세팅

In [ ]:
# 기본 세팅

# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
import optuna

# 경고 안보이게 하기
import warnings
from scipy.linalg import LinAlgWarning
warnings.filterwarnings("ignore", category=LinAlgWarning)

# 한글 폰트
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 파일 불러오기
test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('train.csv')



# 이번 미션 평가지표 RMSLE
def rmsle(y_true, y_pred, convertExp=True):
    if convertExp:
        y_true = np.exp(y_true)
        y_pred = np.exp(y_pred)
    return np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2)))


# 1. EDA

## - 기본 정보 확인

In [ ]:
# 기본 정보 확인
train_df.info()

In [ ]:
# 기본 정보 확인
test_df.info()

In [ ]:
# 기본 정보 확인
train_df.head()

In [ ]:
# 기본 정보 확인
test_df.head()

In [ ]:
# 기본 정보 확인
train_df.describe(include='all')

In [ ]:
# 기본 정보 확인
test_df.describe(include='all')

## - 결측값 확인

In [ ]:
# 결측값 확인
train_df.isna().sum()

In [ ]:
# 결측값 확인
test_df.isna().sum()

## - 중복값 확인

In [ ]:
# 중복값 확인
train_df[train_df.duplicated(keep='first')]

In [ ]:
# 중복값 확인
test_df[test_df.duplicated(keep='first')]

- train 데이터와 test 데이터 모두 결측값, 중복값은 없음.

이상치는 데이터 전처리에서 발견시 처리예정




# 2. 데이터 전처리

### 수치형 데이터

In [ ]:
# 수치형 데이터 (하드 코딩 안하기)
# 목표 변수 count와 이를 구성하는 casual, registered는 drop

num_features = (
    train_df
    .select_dtypes(include='number')
    .loc[:, lambda x: x.nunique() > 10]
    .drop(columns=['casual', 'registered', 'count'], errors='ignore')
    .columns
)

num_features

In [ ]:
# 시각화
for col in num_features:
    plt.figure(figsize=(3,3))
    sns.histplot(test_df[col], kde=True)
    plt.title(col)
    plt.show()

- windspeed는 분포가 치우쳐져 있어서 안정화를 위해 로그 변환
- 나머지는 표준화 진행

In [ ]:
# 목표변수 시각화
plt.figure(figsize=(3,3))
sns.histplot(train_df['count'], kde=True)
plt.title('Count')
plt.show()

- count(목표변수)도 로그 변환

In [ ]:
# 로그 변환
train_df['windspeed'] = np.log1p(train_df['windspeed'])
train_df['count'] = np.log1p(train_df['count'])
test_df['windspeed'] = np.log1p(test_df['windspeed'])

# 표준화
standard_scaler = StandardScaler()
standard_features = ['temp', 'atemp', 'humidity']

train_df[standard_features] = standard_scaler.fit_transform(train_df[standard_features])
test_df[standard_features] = standard_scaler.transform(test_df[standard_features])

### 범주형 데이터

In [ ]:
# 범주형 데이터
cat_features = ['season', 'holiday', 'workingday', 'weather']

In [ ]:
# 각 데이터 고유값 확인
print(train_df['season'].value_counts())
print(train_df['holiday'].value_counts())
print(train_df['workingday'].value_counts())
print(train_df['weather'].value_counts())

In [ ]:
# weather 4인 데이터량이 부족하여, 제외하기
train_df = train_df[train_df['weather'] != 4]
train_df['weather'].value_counts()

In [ ]:
# 인코딩
train_encoded_df = pd.get_dummies(train_df, columns=cat_features, drop_first=True)
test_encoded_df = pd.get_dummies(test_df, columns=cat_features, drop_first=True)

In [ ]:
# train과 test의 컬럼 불일치를 방지

train_encoded_df, test_encoded_df = train_encoded_df.align(test_encoded_df, join='left', axis=1, fill_value=0)

### 날짜형 데이터

In [ ]:
# 날짜형 데이터 타입 변환
train_encoded_df['datetime'] = pd.to_datetime(train_encoded_df['datetime'])
test_encoded_df['datetime'] = pd.to_datetime(test_encoded_df['datetime'])

In [ ]:
# 날짜형 데이터 열 추가

train_encoded_df['year'] = train_encoded_df['datetime'].dt.year
train_encoded_df['month'] = train_encoded_df['datetime'].dt.month
train_encoded_df['day'] = train_encoded_df['datetime'].dt.day
train_encoded_df['hour'] = train_encoded_df['datetime'].dt.hour
train_encoded_df['weekday'] = train_encoded_df['datetime'].dt.weekday

test_encoded_df['year'] = test_encoded_df['datetime'].dt.year
test_encoded_df['month'] = test_encoded_df['datetime'].dt.month
test_encoded_df['day'] = test_encoded_df['datetime'].dt.day
test_encoded_df['hour'] = test_encoded_df['datetime'].dt.hour
test_encoded_df['weekday'] = test_encoded_df['datetime'].dt.weekday  # 시작 요일 : 월(0)

### 분석용 자료 생성

위에서 데이터를 로그변환과 표준화 처리하였기에,

1. 이를 재변환해서 데이터 분석 하고, 이를 다시 변환한 값을 모델링 하는 것보다

2. 변환한 값 자체의 해석한 내용을 표기하는 것보다

-> 분석용 자료를 만들어서 분석하는게 낫다고 판단.



In [ ]:
# 분석용 자료 불러오기
analysis_df = pd.read_csv('train.csv')

analysis_df['datetime'] = pd.to_datetime(analysis_df['datetime'])

analysis_df['year'] = analysis_df['datetime'].dt.year
analysis_df['month'] = analysis_df['datetime'].dt.month
analysis_df['day'] = analysis_df['datetime'].dt.day
analysis_df['hour'] = analysis_df['datetime'].dt.hour
analysis_df['weekday'] = analysis_df['datetime'].dt.weekday

### 적용된 데이터 확인

In [ ]:
# 적용된 데이터 확인하기
train_encoded_df.head()

In [ ]:
# 적용된 데이터 확인하기
test_encoded_df.head()

In [ ]:
# 적용된 데이터 확인하기
train_encoded_df.info()

In [ ]:
# 적용된 데이터 확인하기
test_encoded_df.info()

In [ ]:
# 적용된 데이터 확인하기
train_encoded_df.describe(include='all')

In [ ]:
# 적용된 데이터 확인하기
test_encoded_df.describe(include='all')

In [ ]:
# 적용된 수치형 데이터 시각화
for col in num_features:
    plt.figure(figsize=(3,3))
    sns.histplot(train_encoded_df[col], kde=True)
    plt.title(col)
    plt.show()

In [ ]:
# 적용된 수치형 데이터 시각화
for col in num_features:
    plt.figure(figsize=(3,3))
    sns.histplot(test_encoded_df[col], kde=True)
    plt.title(col)
    plt.show()

In [ ]:
plt.figure(figsize=(3,3))
sns.histplot(train_encoded_df['count'], kde=True)
plt.title('Count')
plt.show()

# 3. 데이터 분석

In [ ]:
# 그룹별 평균을 barplot으로 시각화하는 패턴 만들기
def barplot_pattern(df, group_col, target='count', title=''):
    mean = df.groupby(group_col)[target].mean()
    sns.barplot(x=mean.index, y=mean.values)
    plt.title(title)
    plt.show()

## 1. 수치형 데이터

### 1. 상관관계 분석

In [ ]:
# 상관관계
corr = train_encoded_df.corr()

corr

In [ ]:
# 히트맵
plt.figure(figsize=(12, 12))
sns.heatmap(corr, annot=True, fmt=".2f", cmap='viridis')

In [ ]:
# 목표 변수 기준으로 상관관계

corr_target_data = corr['count'].sort_values(ascending=False)
corr_target_data

상관관계 높은 순

- registered : 등록된 사용자 수

- casual : 등록되지 않은 사용자 수

 **** count는 registered와 casual의 합이므로 상관관계가 높게 나오는 것은 구조적으로 당연한 것

- hour : 시간대

- temp : 실측온도

- atemp : 체감온도

- humidity : 습도 (음의 상관관계)




###  2. 시간대별 대여량 분석

In [ ]:
# 시간대별 평균 대여량

analysis_df['hour'] = analysis_df['datetime'].dt.hour

hour_mean = analysis_df.groupby('hour', observed=False)['count'].mean()
hour_mean


In [ ]:
# 시각화
barplot_pattern(analysis_df, 'hour', target='count', title='시간대별 자전거 이용량')

- 상대적으로 출퇴근 시간에 이용자들이 많은 것을 알 수 있다.

### 3. 온도별 대여량 분석

In [ ]:
# 실제 온도 구간화
analysis_df['temp_group'] = pd.cut(
    analysis_df['temp'],
    bins=[-10, 0, 10, 20, 30, 40],
    labels=['0이하', '0~10', '10~20', '20~30', '30이상']
)

temp_mean = analysis_df.groupby('temp_group', observed=False)['count'].mean()
temp_mean

In [ ]:
# 시각화
barplot_pattern(analysis_df, 'temp_group', target='count', title="실제온도별 자전거 이용량")

In [ ]:
# 체감 온도 구간화
analysis_df['atemp_group'] = pd.cut(
    analysis_df['atemp'],
    bins=[-10, 0, 10, 20, 30, 40],
    labels=['0이하', '0~10', '10~20', '20~30', '30이상']
)

atemp_mean = analysis_df.groupby('atemp_group', observed=False)['count'].mean()
atemp_mean

In [ ]:
# 시각화
barplot_pattern(analysis_df, 'atemp_group', target='count', title="체감온도별 자전거 이용량")

- 실제 온도와 체감 온도가 이용 평균에 미치는 영향은 비슷함 -> 모델 학습시 한개만 적용 예정
- 온도가 낮을 때보다 높을 때, 이용자가 많은 것을 확인

### 4. 습도별 대여량 분석

In [ ]:
# 습도 구간화
analysis_df['humidity_group'] = pd.cut(
    analysis_df['humidity'],
    bins=[0, 30, 60, 100],
    labels=['습도 낮음', '습도 중간', '습도 높음']
)

humidity_mean = analysis_df.groupby('humidity_group', observed=False)['count'].mean()
humidity_mean

In [ ]:
# 시각화
barplot_pattern(analysis_df, 'humidity_group', target='count', title="습도별 자전거 이용량")

- 상대적으로 습도가 낮을 때, 이용자가 많은 것을 확인

### 5. 연도별 대여량 분석

In [ ]:
# 연도별 평균 대여량

year_mean = analysis_df.groupby('year', observed=False)['count'].mean()
year_mean

In [ ]:
# 시각화
barplot_pattern(analysis_df, 'year', target='count', title="연도별 자전거 이용량")

-> 2012년에 이용자들이 더 많았음.

### 6. 월별 대여량 분석

In [ ]:
# 월별 평균 대여량

month_mean = analysis_df.groupby('month', observed=False)['count'].mean()
month_mean

In [ ]:
# 시각화
barplot_pattern(analysis_df, 'month', target='count', title="월별 자전거 이용량")

- 겨울보다 여름~가을에 상대적으로 이용자가 많음.

### 7. 풍속별 대여량 분석

In [ ]:
# 추세선 시각화
sns.regplot(
    data=analysis_df,
    x='windspeed',
    y='count',
    order=2,
    scatter_kws={'alpha':0.2}
)

- 바람이 거의 없을 때보다 바람이 적당할 때 대여량이 많음

- 풍속이 강해지면 대여량이 줄어듬

## 2. 범주형 데이터

수치형 변수 중 day는 평일과 휴일 대여량 비교하는 것에 속해있으므로 분석 예정.

범주형 변수 중 season은 월별 대여량에 속해있으므로 분석 제외.


### 1. 날씨별 대여량 분석

In [ ]:
# 날씨별 대여량
weather_mean = analysis_df.groupby('weather', observed=False)['count'].mean()
weather_mean

In [ ]:
# weather 4인 데이터량이 부족하여 drop하기
analysis_df = analysis_df[analysis_df['weather'] != 4]
analysis_df['weather'].value_counts()

# 날씨별 대여량
weather_mean = analysis_df.groupby('weather', observed=False)['count'].mean()
weather_mean

In [ ]:
# 시각화
barplot_pattern(analysis_df, 'weather', target='count', title="날씨별 자전거 이용량")

- 날씨가 좋지 않을때보다 날씨가 좋을때 대여량이 높음.

- 폭우/ 폭설시에는 이용을 거의 안함.

In [ ]:
# 바람에 대하여 교란 변수(날씨)를 통제하여 시각화
sns.lmplot(
    data=analysis_df,
    x='windspeed',
    y='count',
    col='weather'
)

- 풍속은 날씨 상태와 상호작용하는 변수로 작용하여, 날씨가 안좋으면 바람과 대여량의 관계는 줄어듬.

### 2. 평일과 휴일별 대여량 분석

In [ ]:
# 평일과 휴일별 평균 대여량
holiday_mean = analysis_df.groupby('holiday', observed=False)['count'].mean()
holiday_mean


In [ ]:
# 시각화
barplot_pattern(analysis_df, 'holiday', target='count', title="평일과 휴일별 자전거 이용량")

- 유의미한 차이 없음

# 4. 데이터 모델링

### 1. 학습 데이터 정리

In [ ]:
# 학습 데이터 정리

X = train_encoded_df.drop(columns=['datetime', 'casual', 'registered', 'count'])
y = train_encoded_df['count']

print(X.shape)
print(y.shape)

# 테스트셋 분리(20 : 80)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=5)

### 2. 다중 선형 회귀

In [ ]:
# 다중 선형 회귀 모델 학습
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# 예측
y_train_linear_pred = linear_model.predict(X_train)
y_valid_linear_pred = linear_model.predict(X_valid)

train_rmsle_linear = rmsle(y_train, y_train_linear_pred)
valid_rmsle_linear = rmsle(y_valid, y_valid_linear_pred)

# RMSLE 평가
print(f"훈련 RMSLE= {train_rmsle_linear:.4f}, 검증 RMSLE= {valid_rmsle_linear:.4f}, 차이= {valid_rmsle_linear-train_rmsle_linear:.4f}")

-> 다중선형회귀의 결과값은 다른 모델과 비교용

### 3. 다항 회귀

In [ ]:
results_poly = []

for i in [2, 3]:

    # 다항 변환
    poly = PolynomialFeatures(i)
    X_train_poly = poly.fit_transform(X_train)
    X_valid_poly = poly.transform(X_valid)

    # 다항회귀 모델 학습
    model = LinearRegression()
    model.fit(X_train_poly, y_train)

    # 예측
    y_train_pred = model.predict(X_train_poly)
    y_valid_pred = model.predict(X_valid_poly)

    # RMSLE 평가
    train_rmsle = rmsle(y_train, y_train_pred)
    valid_rmsle = rmsle(y_valid, y_valid_pred)

    # 결과 저장
    results_poly.append({
        'degree': i,
        'train_rmsle': train_rmsle,
        'valid_rmsle': valid_rmsle
    })

    print(f'차수={i} | 훈련 RMSLE={train_rmsle:.4f}, 검증 RMSLE={valid_rmsle:.4f}, 차이 = {valid_rmsle-train_rmsle:.4f}')

-> 2차는 다중 선형회귀 모델보다 좋아졌고, 훈련 결과와 검증 결과의 차이가 거의 없음

-> 3차는 모델은 더 좋아졌으나, 훈련 결과와 검증 결과가 차이가 나기 시작

-> 4차 이후는 모델 복잡도가 증가해 과적합 가능성이 높아, 실험 대상에서 제외

### 3. 로지스틱회귀

현재 회귀문제이므로 생략

### 4. 정규화

#### 1. L2 정규화

In [ ]:

results_l2 = []

for i in [2, 3]:
    for a in [0.1, 1, 10, 50, 100]:

        # 다항 변환
        poly_L2 = PolynomialFeatures(i)
        X_train_poly_L2 = poly_L2.fit_transform(X_train)
        X_valid_poly_L2 = poly_L2.transform(X_valid)

        # Ridge 모델
        ridge_model = Ridge(alpha=a, max_iter=5000)
        ridge_model.fit(X_train_poly_L2, y_train)

        # 예측
        y_train_pred_L2 = ridge_model.predict(X_train_poly_L2)
        y_valid_pred_L2 = ridge_model.predict(X_valid_poly_L2)

        # RMSLE 계산
        train_rmsle = rmsle(y_train, y_train_pred_L2)
        valid_rmsle = rmsle(y_valid, y_valid_pred_L2)

        # 결과 저장
        results_l2.append({
            'degree': i,
            'alpha': a,
            'train_rmsle': train_rmsle,
            'valid_rmsle': valid_rmsle,
        })

        print(f'차수={i}, 알파={a}, train={train_rmsle:.4f}, valid={valid_rmsle:.4f}, 차이={valid_rmsle-train_rmsle:.4f}')


#### 2. L1 정규화

In [ ]:
results_l1 = []

def L1check(i,a):
    # 다항 변환
    poly_L1 = PolynomialFeatures(i)
    X_train_poly_L1 = poly_L1.fit_transform(X_train)
    X_valid_poly_L1 = poly_L1.transform(X_valid)

    # Lasso 모델 (L1)
    lasso_model = Lasso(alpha=a, max_iter=10000, tol=1e-2)
    lasso_model.fit(X_train_poly_L1, y_train)

    # 예측
    y_train_pred_L1 = lasso_model.predict(X_train_poly_L1)
    y_valid_pred_L1 = lasso_model.predict(X_valid_poly_L1)

    # RMSLE 계산
    train_rmsle = rmsle(y_train, y_train_pred_L1)
    valid_rmsle = rmsle(y_valid, y_valid_pred_L1)

    results_l1.append({
        'degree': i,
        'alpha': a,
        'train_rmsle': train_rmsle,
        'valid_rmsle': valid_rmsle
    })

    print(f'차수={i}, 알파={a}, train={train_rmsle:.4f}, valid={valid_rmsle:.4f}, 차이={valid_rmsle-train_rmsle:.4f}')


for i in [2, 3]:
    if i == 2:
        for a in [0.1, 1, 10, 50, 100]:
            L1check(i, a)
    else:
        for a in [10, 50, 100]:   # L2 결과를 보고, L1 에서의 3차는 큰 알파값만 실험함.
            L1check(i,a)


## 5. 모델 선정

In [ ]:
# 다중 선형 회귀 DF화
linear_df = pd.DataFrame([{
    'model': 'Linear',
    'degree': 1,
    'regularization': 'None',
    'alpha': 'None',
    'train_rmsle': train_rmsle_linear,
    'valid_rmsle': valid_rmsle_linear
}])

# 디항 선형 회귀 DF화
poly_df = pd.DataFrame(results_poly)
poly_df['model'] = 'Polynomial'
poly_df['regularization'] = 'None'
poly_df['alpha'] = 'None'

# L2 DF화
ridge_df = pd.DataFrame(results_l2)
ridge_df['model'] = 'Polynomial'
ridge_df['regularization'] = 'L2'

# L1 DF화
lasso_df = pd.DataFrame(results_l1)
lasso_df['model'] = 'Polynomial'
lasso_df['regularization'] = 'L1'

# 데이터 합치기
all_results = pd.concat([linear_df, poly_df, ridge_df, lasso_df], ignore_index=True)

# 훈련rmsle와 검증rmsle 차이 추가
all_results["diff"] = all_results['valid_rmsle']-all_results['train_rmsle']

# 정렬하기
all_results[['model', 'regularization', 'degree', 'alpha', 'train_rmsle', 'valid_rmsle', 'diff']].sort_values('valid_rmsle')


    -> 선택 모델 : 다항회귀 모델 + 3차 + 알파값100 + L2 정규화

## 6. 하이퍼 파타미터 탐색 (Optuna)

In [ ]:
# Optuna
def objective(trial):

    # 탐색할 하이퍼파라미터
    degree = trial.suggest_int('degree', 2, 3)
    alpha = trial.suggest_float('alpha', 0.01, 100, log=True)

    # 다항 변환
    poly = PolynomialFeatures(degree)
    X_train_poly = poly.fit_transform(X_train)
    X_valid_poly = poly.transform(X_valid)

    # Ridge 모델
    model = Ridge(alpha)
    model.fit(X_train_poly, y_train)

    # 예측
    y_valid_pred = model.predict(X_valid_poly)

    return rmsle(y_valid, y_valid_pred)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print(f'study.best_params: {study.best_params}')
print(f'study.best_value: {study.best_value}')


GridSearch는 전수 탐색이라 비효율적이라 판단하여 Optuna를 사용함.

처음부터 train데이터와 test데이터가 분리되어 있어

OptunaSearchCV 대신 create_study 기반으로 objective 함수를 정의하여 사용했음.

# 5. 최종 test

In [ ]:
# test 데이터 정리
X_test = test_encoded_df.select_dtypes(exclude=['datetime64'])

# train 기준으로 test 컬럼 맞추기
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# 1train + valid 합치기 (최종 학습용)
X_train_final = np.vstack([X_train, X_valid])
y_train_final = np.concatenate([y_train, y_valid])

# 최종 파라미터 - Optuna 결과 반영
final_degree = study.best_params['degree']
final_alpha = study.best_params['alpha']

# 다항 변환
poly = PolynomialFeatures(final_degree)
X_train_final_poly = poly.fit_transform(X_train_final)
X_test_poly = poly.transform(X_test)

# 최종 L2 학습
final_model = Ridge(alpha=final_alpha)
final_model.fit(X_train_final_poly, y_train_final)

# test 예측
y_test_pred_log = final_model.predict(X_test_poly)
y_test_pred = np.expm1(y_test_pred_log)

# 음수는 0으로 클리핑
y_test_pred = np.maximum(0, y_test_pred)

# 결과 확인

print("음수 개수:", (y_test_pred < 0).sum())
print("NaN 개수:", np.isnan(y_test_pred).sum())
print("inf 개수:", np.isinf(y_test_pred).sum())

로그변환을 이용하여 일부 음수 예측이 발생해 물리적으로 불가능한 값은 0으로 보정하였음.

In [ ]:
# 기존 test_df 확인하기
test_df.head()

In [ ]:
# test_df에 최종 예측값 추가하기
test_df['pred_count'] = y_test_pred

In [ ]:
# 최종 결과 확인
test_df.head()

In [ ]:
# 학습데이터의 목표변수와 모델을 이용하여 찾은 최종 예측값 분포 확인하기

sns.kdeplot(np.expm1(train_df['count']), label='학습용', fill=True)
sns.kdeplot(test_df['pred_count'], label='최종 예측', fill=True)
plt.legend()
plt.title("학습 데이터와 예측 데이터의 분포 비교")
plt.show()

# 6. 결론

다중선형회귀를 기준으로 다항회귀와 정규화를 적용하여,

valid_rmsle 기준으로 Optuna를 활용해

    다항회귀 3차 + L2 Ridge(알파값=100) 모델을 최종 선택했다.

초반에 나눴던 train데이터와 valid데이터를 통합한 뒤

선택한 모델로 재학습하고 test 데이터에 대한 최종 예측을 수행하였고,

음수값이 일부 발생하였으나, 물리적으로 공유 자전거 수요에서 음수값은 나올 수 없으므로

0으로 보정하고, 결측값이나 무한대값이 없음을 확인하였다.

최종적으로 학습 데이터와 예측 데이터의 분포를 비교하니, 비슷한 분포를 보였다.